In [1]:
# Standard modules
import pdb
import os
import sys
import pandas as pd
from astropy.io import fits
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Run script to generate maps

In [2]:
run gen_V22_maps.py

Generate the map for MIPS24...
Set World Coordinates System...
Convolve the map by the beam...
Write D:\simulations\sides\maps\pySIDES_from_original_V22_MIPS24_smoothed_Jy_beam.fits...
Generate the map for PACS100...
Set World Coordinates System...
Convolve the map by the beam...
Write D:\simulations\sides\maps\pySIDES_from_original_V22_PACS100_smoothed_Jy_beam.fits...
Generate the map for PACS160...
Set World Coordinates System...
Convolve the map by the beam...
Write D:\simulations\sides\maps\pySIDES_from_original_V22_PACS160_smoothed_Jy_beam.fits...
Generate the map for SPIRE250...
Set World Coordinates System...
Convolve the map by the beam...
Write D:\simulations\sides\maps\pySIDES_from_original_V22_SPIRE250_smoothed_Jy_beam.fits...
Generate the map for SPIRE350...
Set World Coordinates System...
Convolve the map by the beam...
Write D:\simulations\sides\maps\pySIDES_from_original_V22_SPIRE350_smoothed_Jy_beam.fits...
Generate the map for SPIRE500...
Set World Coordinates System..

# Import simulated catalog

In [2]:
# Import table with astropy
CATSPATH = 'D:\\simulations\\sides\\'
path_sides = os.path.join(CATSPATH, "pySIDES_from_original.fits")
if os.path.isfile(path_sides):
    print('Open ',path_sides)
    with fits.open(path_sides) as hdul:
        hdul.verify('fix')
        sides_catalog = hdul[1].data
else:
    print(path_sides, ' not found')

Open  D:\simulations\sides\pySIDES_from_original.fits


In [3]:
sides_df = pd.DataFrame(sides_catalog)

In [7]:
sides_df.keys()

Index(['redshift', 'ra', 'dec', 'Mhalo', 'Mstar', 'qflag', 'SFR', 'issb', 'mu',
       'Dlum', 'Umean', 'LIR', 'S24', 'S70', 'S100', 'S160', 'S250', 'S350',
       'S500', 'S850', 'S1100', 'S2000', 'LFIR', 'SMIPS24', 'SPACS70',
       'SPACS100', 'SPACS160', 'SSPIRE250', 'SSPIRE350', 'SSPIRE500',
       'SNIKA1200', 'SNIKA2000', 'LprimCO10', 'ICO10', 'ICO21', 'ICO32',
       'ICO43', 'ICO54', 'ICO65', 'ICO76', 'ICO87', 'LCII_Lagache',
       'ICII_Lagache', 'LCII_de_Looze', 'ICII_de_Looze', 'ICI10', 'ICI21'],
      dtype='object')

### Define columns to keep and write CSV

In [37]:
sfg_df = sides_df.loc[sides_df.qflag==False]
sfg_df.head()

,redshift,ra,dec,Mhalo,Mstar,qflag,SFR,issb,mu,Dlum,...,ICO54,ICO65,ICO76,ICO87,LCII_Lagache,ICII_Lagache,LCII_de_Looze,ICII_de_Looze,ICI10,ICI21
0,0.027082,1.386314,0.578664,9.755162e+11,1.166001e+10,False,2.131869,False,0.995326,122.324178,...,60.299789,41.618616,26.379773,14.910044,6.255425e+07,2162.181349,4.323390e+07,1494.375393,35.256016,41.052132
2,0.028948,0.653517,1.053531,3.458112e+11,4.891516e+09,False,0.921371,False,1.005174,130.932469,...,37.693771,24.830785,15.271163,8.400091,5.801900e+06,177.092316,1.751020e+07,534.466683,17.568185,16.497085
4,0.028611,0.787421,0.940826,3.362242e+11,1.672992e+09,False,0.268678,False,0.993735,129.376111,...,7.411665,4.853775,2.973266,1.629442,2.297588e+06,70.986414,6.843385e+06,211.433599,2.785102,3.118200
5,0.030565,1.024233,1.341723,2.921091e+11,4.972536e+08,False,0.110024,False,1.006393,138.410613,...,2.102942,1.390806,0.857629,0.472907,2.741590e+05,7.509246,5.356292e+05,14.670944,1.881445,3.024126
6,0.027382,1.168631,0.542636,2.262832e+11,6.630631e+08,False,0.025710,False,0.995309,123.706599,...,3.420046,2.312872,1.447210,0.808680,3.839952e+04,1.298134,3.121675e+05,10.553132,2.352570,0.870817


In [4]:
sides_df[['ra','dec','Mstar','qflag','SFR','LIR','S24','S100','S160','S250','S350','S500','S850']]

,ra,dec,Mstar,qflag,SFR,LIR,S24,S100,S160,S250,S350,S500,S850
0,1.386314,0.578664,1.166001e+10,False,2.131869,2.131869e+10,0.031762,0.978360,0.992486,0.478294,0.200496,0.070784,0.013884
1,0.454959,1.145274,2.612670e+10,True,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.653517,1.053531,4.891516e+09,False,0.921371,9.213706e+09,0.016423,0.338908,0.301663,0.133905,0.053814,0.018472,0.003558
3,0.931048,0.651234,1.876074e+09,True,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.787421,0.940826,1.672992e+09,False,0.268678,2.686784e+09,0.004899,0.099181,0.086771,0.038089,0.015216,0.005203,0.001000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5584993,1.372015,0.020294,1.499852e+08,True,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5584994,1.009508,0.299231,1.022215e+07,True,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5584995,1.084438,0.680461,3.201675e+07,True,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5584996,1.167128,1.148331,2.027734e+07,True,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
path_sides_csv = os.path.join(os.environ['CATSPATH'], "cosmos","sides_full.csv")
sides_df[['redshift','ra','dec','Mstar','qflag','SFR','LIR','SMIPS24','S100','S160','SSPIRE250','SSPIRE350','SSPIRE500','S850']].dropna().to_csv(path_sides_csv)